This capstone project is aimed at explore,segment and cluster the neighborhoods in the city of Toronto.
Libraries used:
Numpy,pandas,Beautiful soup 
Foursquare API

In [1]:

#importing the required libraries 
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

import pip

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  --> 2019.9.11-py36_0  conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1d-h7b6447c_3 --> 1.1.1c-h516909a_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Libraries imported.


In [2]:
conda install beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1d-h7b6447c_3


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [3]:
# importing libraries
from bs4 import BeautifulSoup
import urllib.request
import re


In [66]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url) # conntect to website

In [67]:
#for parsing the html the page needs to be passed through beautiful soup
soup = BeautifulSoup(page, 'html.parser')
#print(soup)

In [68]:
htmltable = soup.find('table', { 'class' : "wikitable sortable"})
#print(htmltable)
# where the dictionary specify unique attributes for the 'table' tag
def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = [td.get_text(strip=True) for td in trs[0].find_all('th')] # header row
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append([td.get_text(strip=True) for td in tr.find_all('td')]) # data row
    return rows

In [69]:
list_table = tableDataText(htmltable)
list_table[:2]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned']]

In [70]:
#transforming the table to the dataframe
import pandas as pd
dftable = pd.DataFrame(list_table[1:], columns=list_table[0])
dftable.head(4)
#dftable.shape


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


In [71]:
#Dropping rows borough is not assigned
dftable = dftable[(dftable.Borough!="Not assigned")]

In [72]:
dftable.head(4)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [73]:
dftable.shape




(211, 3)

In [74]:
#Merging rows with same postcode and different neighborhoods
df = dftable.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [75]:
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [76]:
df1=df[df.Neighbourhood=='Not assigned']


In [77]:
df1

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [78]:
#Replacing the not assigned neighbourhood names to their borough names
df.Neighbourhood[df.Neighbourhood=='Not assigned']=df.Borough

In [79]:
df[df.Borough=="Queen's Park"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [80]:
df.shape

(103, 3)